<img src="./images/TODO.svg"/>

[Photo](https://en.wikipedia.org/wiki/Kalaripayattu#/media/File:Kalarippayattu.jpg) by Leelavathy B.M / [CC BY-SA 3.0](https://creativecommons.org/licenses/by-sa/3.0/) / Not actually Kung Fu

TODO --> reemplazar con nuestra propia imagen

In [4]:
# TODO: find way to HIDE (but run) this block.
# Stubs so the examples works
class DeathRay(object):
    def __init__(self, *args, **kwargs): pass
    def vaporize(self, *args, **kwargs): pass
class TimeMachine(object):
    def go(self, *args, **kwargs):
        pass

# The Cyborg class

In [5]:
import time

class Cyborg(object):

    def __init__(self, name):
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()

    def travel(self, destination, year):
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    def attack(self, target):
        self.weapon.vaporize(target)

This is our class, and we want to log the calls to _every_ method.

# Attempt 1: print() calls everywhere

No point in denying it: this is what many of us _would_ do if not supervised by an adult. 

In [6]:
import time

class Cyborg(object):

    def __init__(self, name):
        print("Creating new Cyborg with name '{}'".format(name))
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()

    def travel(self, destination, year):
        print("Travelling to {} and year {}".format(destination, year))
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    def attack(self, target):
        print("Attacking {}".format(target))
        self.weapon.vaporize(target)

"The most effective debugging tool is still careful thought, coupled with judiciously placed print statements"  [Brian W. Kernighan](https://en.wikipedia.org/wiki/Brian_Kernighan).

In [7]:
robot = Cyborg('T-1000')
robot.travel('Los Angeles', 1995)
robot.attack('Sarah Connor')
robot.attack('John Connor')

Creating new Cyborg with name 'T-1000'
Travelling to Los Angeles and year 1995
Attacking Sarah Connor
Attacking John Connor


# Attempt 2: the `logging` module

In [8]:
import time
import logging
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.DEBUG)

class Cyborg(object):

    def __init__(self, name):
        logging.info("Creating new Cyborg with name '%s'", name)
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()

    def travel(self, destination, year):
        logging.info("Travelling to %s and year %s", destination, year)
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    def attack(self, target):
        logging.info("Attacking %s", target)
        self.weapon.vaporize(target)

In [9]:
robot = Cyborg('T-1000')
robot.travel('Los Angeles', 1995)
robot.attack('Sarah Connor')
robot.attack('John Connor')

2017-09-10 11:26:12,687 Creating new Cyborg with name 'T-1000'
2017-09-10 11:26:12,688 Travelling to Los Angeles and year 1995
2017-09-10 11:26:12,939 Attacking Sarah Connor
2017-09-10 11:26:12,940 Attacking John Connor


# Problem: there're logging calls everywhere

- We must add manually the call to e.g. `logging.info()` to every method.
- Whoever adds a new method in the future might forget to do it.
- What if we had three _hundred_ methods, instead of three?
- Also, copy-pasting function calls is boring.

In [10]:
import time
import logging
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.DEBUG)

class Cyborg(object):

    def __init__(self, name):
        logging.info("Creating new Cyborg with name '%s'", name)
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()
        self.alive = True

    def travel(self, destination, year):
        logging.info("Travelling to %s and year %s", destination, year)
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    def attack(self, target):
        logging.info("Attacking %s", target)
        self.weapon.vaporize(target)
        
    def selfdestroy(self):
        self.alive = False

We added `Cyborg.selfdestroy()`, but forgot to add the logging call.

# Detour: Decorators Crash Course

What are decorators and what are the useful for?

# TODO IMAGE

# First-class objects

> "A first class object is an entity that can be dynamically created, destroyed, passed to a function, returned as a value, and have all the rights as other variables in the programming language have" --- [StackOverflow](https://stackoverflow.com/a/245208)

* Functions are first-class objects.
* Classes are first-class objects too.
* In fact, in Python _everything_ is a first-class object.

This mean that if we take e.g. a function, we can...


## Assign it to a variable

Take a function, for example. We can assign it to a variable like any other object.

In [11]:
numbers = [3, 5, 7, 1]
print(max(numbers))

7


In [12]:
func = max
print(func(numbers))  # same as calling max()

7


No parentheses because we are not calling the function.

## Pass it as argument

... _to_ another function:

In [13]:
def call(func, values):
    return func(values)

print(call(max, numbers))
print(call(min, numbers))

7
1


This is used in real life for [callbacks][1], among others.

[1]: https://en.wikipedia.org/wiki/Callback_(computer_programming)

## Return it

... _from_ another function:

In [14]:
import random                                                               

func = random.choice([max, min])
print("Function:", func)
print("Result:", func(numbers))

Function: <built-in function min>
Result: 1


## Nested functions

Define it within another function, and return it!

In [15]:
def get_power_function(exponent):
    """Returns a function to compute the exponent-th power."""
    
    def power(n):
        return n ** exponent
    return power

square = get_power_function(2)
cube   = get_power_function(3)
n = 4

print("Number:", 4)
print("Square:", square(n))
print("Cube  :", cube(n))

Number: 4
Square: 16
Cube  : 64


# So, decorators...

... are "wrappers" that let us execute code _before_ and _after_ the function that they decorate without modifying the function itself. We:

* Take the function as an argument.
* Add some behaviour, wrapping it in a new function.
* Return (and later use) the new function.

In [16]:
import time

def measure_time(func):
    """A decorator for measuring the execution time of a function."""

    def wrapped(*args):
        tstart = time.time()
        result = func(*args)
        tend = time.time()
        tdelta = tend - tstart
        print("Function call took {} seconds".format(tdelta))
        return result
    
    return wrapped

Note: no `**kwargs` for simplicity's sake.

A more Pythonic approach, by the way, would be to use [finally](https://docs.python.org/3/tutorial/errors.html#defining-clean-up-actions):

In [17]:
import time

def measure_time(func):
    """A decorator for measuring the execution time of a function."""

    def wrapped(*args):
        try:
            tstart = time.time()
            return func(*args)
        finally:
            tend = time.time()
            tdelta = tend - tstart
            print("Function call took {} seconds".format(tdelta))

    return wrapped

No need to store the result in a variable to return it later.

Let's now decorate something:

In [18]:
def square_everything(numbers):
    """Return the square of all the numbers."""
    
    result = []
    for n in numbers:
        result.append(n ** 2)
    return result

func = measure_time(square_everything)
print(func(numbers))

Function call took 2.86102294921875e-06 seconds
[9, 25, 49, 1]


# Missing attributes

A problem with our decorated function is that we lose attributes such as the name or docstring.

In [19]:
def square_everything(numbers):
    """Return the square of all the numbers."""
    
    result = []
    for n in numbers:
        result.append(n ** 2)
    return result

print("Name:", square_everything.__name__)
print("Docstring:", square_everything.__doc__)

Name: square_everything
Docstring: Return the square of all the numbers.


However...

In [20]:
func = measure_time(square_everything)

print("Name:", func.__name__)
print("Docstring:", func.__doc__)

Name: wrapped
Docstring: None


This is unfortunate, as these are great for e.g. debugging.

# functools.wraps()

[`wraps()`](https://docs.python.org/3/library/functools.html#functools.wraps) function allows us to overwrite the function attributes (`__name__`, `__doc__`, `__module__`, etc) attributes of the wrapper function with those of the _original_ function.

In [21]:
import functools
import time

def measure_time(func):
    """A decorator for measuring the execution time of a function."""

    @functools.wraps(func)  # note this
    def wrapped(*args):
        try:
            tstart = time.time()
            return func(*args)
        finally:
            tend = time.time()
            tdelta = tend - tstart
            print("Function call took {} seconds".format(tdelta))

    return wrapped

In this manner, the changes to the function are transparent.

In [22]:
def square_everything(numbers):
    """Return the square of all the numbers."""
    
    result = []
    for n in numbers:
        result.append(n ** 2)
    return result

func = measure_time(square_everything)

print("Name:", func.__name__)
print("Docstring:", func.__doc__)

Name: square_everything
Docstring: Return the square of all the numbers.


# Python's Decorator Syntax

But we don't want to call our function `func`. Let's keep the original name:

In [23]:
def square_everything(numbers):
    """Return the square of all the numbers."""
    
    result = []
    for n in numbers:
        result.append(n ** 2)
    return result

square_everything = measure_time(square_everything)
print(square_everything(numbers))

Function call took 3.5762786865234375e-06 seconds
[9, 25, 49, 1]


Instead of...

In [24]:
square_everything = measure_time(square_everything)

... we can apply a decorator using this shortcut:

In [25]:
@measure_time
def square_everything(numbers):
    result = []
    for n in numbers:
        result.append(n ** 2)
    return result

print(square_everything(numbers))

Function call took 4.0531158447265625e-06 seconds
[9, 25, 49, 1]


That is: apply `measure_time` to `square_everything` and store it in `square_everything`. We're effective replacing it with the updated, wrapped version.

# There's much more

* We can chain decorators, applying 2+ to the same function.
* We can have decorators that take arguments. 
* A closely-related concept are _closures_.

But this is enough for now.

### Recommended readings:

* [Decorator Basics](https://stackoverflow.com/a/1594484) on Stack Overflow.
* [The closures that moved Spielberg](https://www.youtube.com/watch?v=rrL3CQNOFRc) at PyConES 2016.

# Going back to our problem...

We were here, with our login calls _everywhere_.

In [26]:
import time
import logging
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.DEBUG)

class Cyborg(object):

    def __init__(self, name):
        logging.info("Creating new Cyborg with name '%s'", name)
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()
        self.alive = True

    def travel(self, destination, year):
        logging.info("Travelling to %s and year %s", destination, year)
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    def attack(self, target):
        logging.info("Attacking %s", target)
        self.weapon.vaporize(target)

# Attempt 3: method decorators

TODO -> aquí hay un salto repentino a los decoradores, y se empiezan a usar directamente. Creo que vendría bien un par de breves slides en las que hacer un crash course de 1 minuto (mejor que nada para la gente no sepa qué ocurre, al menos que entiendan el concepto)

In [27]:
import functools

def log(func):
    @functools.wraps(func)
    def wrapped(*args):  # omit **kwargs for simplicity
        logging.info('Called %s, args=%s', func.__name__, args)
        return func(*args)
    return wrapped

@log
def cube(n):
    """Return the third power of 'n'."""
    return n ** 3

print(cube(2))

2017-09-10 11:26:13,328 Called cube, args=(2,)


8


So going back to our `Cyborg` class...

In [28]:
import time
import logging

def log(func):
    @functools.wraps(func)
    def wrapped(*args):  # omit **kwargs for simplicity
        # args[1:] so that we don't print 'self'
        logging.info('Called %s%s', func.__name__, args[1:])
        return func(*args)
    return wrapped


class Cyborg(object):
    
    @log
    def __init__(self, name):
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()

    @log
    def travel(self, destination, year):
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    @log
    def attack(self, target):
        self.weapon.vaporize(target)

In [29]:
robot = Cyborg('T-1000')
robot.travel('Los Angeles', 1995)
robot.attack('Sarah Connor')
robot.attack('John Connor')

2017-09-10 11:26:13,379 Called __init__('T-1000',)
2017-09-10 11:26:13,380 Called travel('Los Angeles', 1995)
2017-09-10 11:26:13,634 Called attack('Sarah Connor',)
2017-09-10 11:26:13,635 Called attack('John Connor',)


# This didn't solve the problem, though

- We have replaced calls to `logging.info()` with calls to `@log`.
- Whoever adds a new method in the future might forget to _decorate_ it.
- We still could have three _hundred_ methods instead of three.
- Also, copy-pasting method decorations is still boring.

# Attempt 4: class decorators

Let a second decorator do the work for us $\rightarrow$ decorate all the methods.



In [30]:
def decorate_all_methods(decorator):
    def wrapped(cls):
        for attr_name in cls.__dict__:
            attr = getattr(cls, attr_name)
            if callable(attr):
                # It's a function, decorate it
                setattr(cls, attr_name, decorator(attr))
        return cls
    return wrapped

In [35]:
@decorate_all_methods(log)
class Cyborg(object):

    def __init__(self, name):
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()

    def travel(self, destination, year):
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    def attack(self, target):
        self.weapon.vaporize(target)

In [36]:
robot = Cyborg('T-1000')
robot.travel('Los Angeles', 1995)
robot.attack('Sarah Connor')
robot.attack('John Connor')

2017-09-10 11:26:51,999 Called __init__('T-1000',)
2017-09-10 11:26:52,001 Called travel('Los Angeles', 1995)
2017-09-10 11:26:52,252 Called attack('Sarah Connor',)
2017-09-10 11:26:52,253 Called attack('John Connor',)


# We did it! Hoo-ray!

In [33]:
@decorate_all_methods(log)
class Cyborg(object):

    def __init__(self, name):
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()

    def travel(self, destination, year):
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    def attack(self, target):
        self.weapon.vaporize(target)

# We _didn't_ solve the problem, though

- We just moved one level up the ladder of abstraction.
- We have replaced decorating methods with decorating _classes_.
- Whoever adds a new _class_ in the future might forget to _decorate_ it.
- What if we had one _hundred_ classes instead of just one?

In [39]:
@decorate_all_methods(log)
class Ninja(object): pass
    # ...
    
class Human(object): pass
    # ...
    
class Terminator(object): pass
    # ...  

# Nested classes

Another limitation is that out decorator will also decorate _nested_ classes, even if that's not what we want.

In [78]:
@decorate_all_methods(log)
class Cyborg(object):

    class Chainsaw(object):
        
        # This method was also decorated.
        def vaporize(self, victim): pass
            # ...            
    
    def __init__(self, name):
        self.name = name
        self.weapon = Cyborg.Chainsaw()

    def attack(self, target):
        self.weapon.vaporize(target)

robot = Cyborg('T-1000')
robot.attack('Sarah Connor')

2017-09-10 12:18:59,296 Called __init__('T-1000',)
2017-09-10 12:18:59,297 Called Chainsaw()
2017-09-10 12:18:59,298 Called attack('Sarah Connor',)


In [ ]:
Android = Human + Robot  # get new class and use it
replicant = Android('Roy Batty')

### Recommended readings:

* [Python metaclasses vs class decorators](https://stackoverflow.com/a/1779404) on Stack Overflow.
* [What are Python metaclasses useful for?](https://stackoverflow.com/a/1779404), also by Alex Martelli.

# Help me, Obi-Wan Kenobi. You're my only hope

There're things that simply _cannot_ be done with class decorators.

In [ ]:
But _metaclasses_ can.

# Classes and instances

* Every object is the instance of a class.
* We can check this with the built-in [`type()`](https://docs.python.org/3/library/functions.html#type).

In [69]:
number = 21
word = "blue"

print(type(number))
print(type(word))

<class 'int'>
<class 'str'>


Let's check our own class too:

In [81]:
class Cyborg(object):

    def __init__(self, name):
        self.name = name
        self.weapon = DeathRay(ammunition=25)
        self.teleporter = TimeMachine()

    def travel(self, destination, year):
        self.teleporter.go(destination, year)
        time.sleep(0.25)  # not instant, but almost

    def attack(self, target):
        self.weapon.vaporize(target)

arnold = Cyborg("T-800")
print(type(arnold))

<class '__main__.Cyborg'>


# Wait, but...

* (1) _Every_ object is an instance of a class.
* (2) We said that classes are _objects_ too.
* From (1) and (2) it follows that classes are instances of... what?

In [82]:
print(type(int))
print(type(str))
print(type(Cyborg))

<class 'type'>
<class 'type'>
<class 'type'>


So all classes are instances of class `type`.

# Down the rabbit hole

And `type` is an instance of...

In [83]:
print(type(type))

<class 'type'>


Everything is an instance of `type`, including _itself_.

# The Most Important Slide

If you're going to remember only one thing, it should be this:

> Instances of a class are to **classes** what classes are to **metaclasses**.

<img src="./images/instance-of.png"/>
TODO: necesitamos nuestro propio diagrama, because of copyright



* In the same way a class defines the creation and behaviour of an instance of that class...
* ... metaclasses allow us to define the creation and behaviour of our **classes**.



# `type()` to define new classes

As we just saw, the built-in [`type()`](https://docs.python.org/3/library/functions.html#type) returns the type on an object.

In [84]:
numbers = [1, 2, 3]
print(type(numbers))

<class 'list'>


However, it can also be called with _three_ arguments to return a **new type** object. [From the docs](https://docs.python.org/3/library/functions.html#type):

> With three arguments, return a new type object. This is essentially a dynamic form of the [`class`](https://docs.python.org/3/reference/compound_stmts.html#class) statement. The *name* string is the class name and becomes the [`__name__`](https://docs.python.org/3/library/stdtypes.html#definition.__name__) attribute; the *bases* tuple itemizes the base classes and becomes the [`__bases__`](https://docs.python.org/3/library/stdtypes.html#class.__bases__) attribute; and the *dict* dictionary is the namespace containing definitions for class body and is copied to a standard dictionary to become the [`__dict__`](https://docs.python.org/3/library/stdtypes.html#object.__dict__) attribute. For example, the following two statements create identical [`type`](https://docs.python.org/3/library/functions.html#type) objects:


In [87]:
class X(object):
    a = 1
    
# The above class is identical to:
X = type('X', (object,), dict(a=1))

# Another example

In [94]:
name = "Cylon"
base = (object,)

# We need 'self', of course.
def attack(self, victims):
    print("Destroy all {}!".format(victims))

body = {'attack': attack}
    
Cylon = type(name, base, body)
centurion = Cylon()
centurion.attack("humans")

Destroy all humans!


# How is this useful?

What's the purpose of creating classes dynamically with `type()`?

* We definitely don't need them _often_.
* However, _sometimes_ is the appropriate solution.
* For example, for GUI programming it's very convenient to define classes on the fly and instantiate widgets with them.

## A real-life example: [SQLAlchemy](https://www.sqlalchemy.org/)

> There is only one way to register a database table with SQLAlchemy: create a `Model` class describing the table (not unlike Django's models). To get SQLAlchemy to recognize a table, a class for that table must be created in some way. Since `sandman` doesn't have any advanced knowledge of the database structure, it can't rely on pre-made model classes to register tables. Rather, it needs to introspect the database and create these classes on the fly. Sound familiar? **Any time you're creating new classes dynamically, `type` is the correct/only choice**.

[Improve Your Python: Metaclasses and Dynamic Classes With Type](https://jeffknupp.com/blog/2013/12/28/improve-your-python-metaclasses-and-dynamic-classes-with-type/), by Jeff Knupp.

# Using metaclasses

# TODO
